# 11번가 crawler

- 출처
    1. Selenium : https://beomi.github.io/2017/02/27/HowToMakeWebCrawler-With-Selenium/
    2. 이미지/동영상 가져오기 : https://blog.naver.com/PostView.nhn?blogId=kiddwannabe&logNo=221358581642&categoryNo=35&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView&userTopListOpen=true&userTopListCount=5&userTopListManageOpen=false&userTopListCurrentPage=1
    3. BeautifulSoap : https://blog.naver.com/PostView.nhn?blogId=kiddwannabe&logNo=221177292446&parentCategoryNo=&categoryNo=35&viewDate=&isShowPopularPosts=false&from=postView

In [10]:
#!pip3 install selenium
#!pip3 install bs4

In [124]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import requests
import json

In [4]:
driver = webdriver.Chrome('//Users/sktelecom/Documents/GitHub/crawlers/chromedriver')

In [5]:
# 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다.
driver.implicitly_wait(3)

In [18]:
# 특정 URL로 브라우저를 켜는 부분. 아디다스 도복 페이지
driver.get('http://www.11st.co.kr/product/SellerProductDetail.tmall?method=getSellerProductDetail&prdNo=2064276476&trTypeCd=21&trCtgrNo=585021&lCtgrNo=1001390&mCtgrNo=1002164')

### Selenium 자체 제공 함수

- URL에 접근하는 api,
    1. get(‘http://url.com’)
- 페이지의 단일 element에 접근하는 api,
    1. find_element_by_name(‘HTML_name’)
    2. find_element_by_id(‘HTML_id’)
    3. find_element_by_xpath(‘/html/body/some/xpath’)
- 페이지의 여러 elements에 접근하는 api 등이 있다.
    1. find_element_by_css_selector(‘#css > div.selector’)
    2. find_element_by_class_name(‘some_class_name’)
    3. find_element_by_tag_name(‘h1’)

위 메소드들을 활용시 HTML을 브라우저에서 파싱해주기 때문에 굳이 Python와 BeautifulSoup을 사용하지 않아도 된다.
하지만 Selenium에 내장된 함수만 사용가능하기 때문에 좀더 사용이 편리한 soup객체를 이용하려면 driver.page_source API를 이용해 현재 렌더링 된 페이지의 Elements를 모두 가져올 수 있다.

In [19]:
page = driver.page_source

In [22]:
#page

In [23]:
soup = BeautifulSoup(page, 'html.parser')

In [47]:
product_name = soup.select('title')[0]

In [48]:
product_price = soup.select('strong.num')[4]

In [108]:
product_info = soup.select('script')

In [109]:
product_info

[<script async="" src="http://www.google-analytics.com/analytics.js" type="text/javascript"></script>,
 <script async="" src="http://www.google-analytics.com/plugins/ua/ec.js" type="text/javascript"></script>,
 <script async="" src="https://www.googleadservices.com/pagead/conversion_async.js" type="text/javascript"></script>,
 <script async="" src="//c.011st.com/js/rake/bundle/rake.bundle-0.0.0.js" type="text/javascript"></script>,
 <script async="" src="//www.googletagmanager.com/gtm.js?id=GTM-P3PMQD&amp;l=GA_dataLayer"></script>,
 <script async="" src="//www.google-analytics.com/analytics.js"></script>,
 <script async="" src="https://connect.facebook.net/signals/config/2299486926950990?v=2.9.5&amp;r=stable"></script>,
 <script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script>,
 <script src="//www.11st.co.kr/html/experiment/experimentMetaInfo.js" type="text/javascript"></script>,
 <script src="//www.11st.co.kr/js/plab/plab-web-sdk-0.9.1.min.js" type="text/javascr

In [49]:
print(product_name)
print(product_price)

<title>주짓수 도복 IBJJF CHAMPION JJ601B 2.0 BURE/ 아디다스(의류) - 11번가</title>
<strong class="num">199,230</strong>


### 위의 string에 rule을 걸어 전처리해주고, 필요 정보만 뽑아 내면 끝!

# Solution 2 : page 자체에서 관리하고 있는 상품 정보 전체를 긁어 와서 전처리
    - <script> </script> 안쪽을 뽑아 오려 하는 경우, script는 html이 아니라서 beautifulsoap으로 뽑아 내기 어려움.
    - 정규식 사용
    - 출처 : https://m.blog.naver.com/21ahn/221329219163

In [110]:
lv3_url = 'http://www.11st.co.kr/product/SellerProductDetail.tmall?method=getSellerProductDetail&prdNo=2064276476&trTypeCd=21&trCtgrNo=585021&lCtgrNo=1001390&mCtgrNo=1002164'

In [117]:
page_source = requests.get(lv3_url).text

In [119]:
matched = re.search(r'var productPrdInfo = (.+)' , page_source, re.S)

In [120]:
matched  #json 형태로 추출됨

<re.Match object; span=(260673, 288334), match='var productPrdInfo = {\n\t\tprdNo            : 20>

In [137]:
json_string = matched.group()

In [138]:
json_string

'var productPrdInfo = {\n\t\tprdNo            : 2064276476, \t\t\t\t\t\t        // 상품번호\n\t\tprdNm            : \'주짓수 도복 IBJJF CHAMPION JJ601B 2.0 BURE/ 아디다스(의류)\', \t\t\t\t\t\t\t// 상품명\n\t\tisPreview        : false,\n        isSmtOptChk\t\t : false,\t\t\t\t\t\t\t// 스마트옵션 검수여부\n        reviewDispYn     : "Y",\n\t\treviewOptDispYn  : "Y",\n\t\tsellerId         : "lotteec11",\t\t\t\t\t\t// 판매자ID\n\t\tselMnbdNo        : 37597600,\n\t\tisMart           : "false",\t\t\t\t\t\t\t\t// 마트상품여부\n\t\tisTour           : false,\t\t\t\t\t\t\t\t// 여행상품여부\n\t\tisSohoPrd        : false,\t\t\t\t\t\t\t// 소호상품여부\n\t\tldispCtgrNo      : \'1001390\',\t\t\t\t\t\t// 대카테고리번호\n\t\tmdispCtgrNo      : \'1002164\',\t\t\t\t\t\t// 중카테고리번호\n\t\tsdispCtgrNo      : \'1007632\',\t\t\t\t\t\t// 소카테고리번호\n\t\tdispCtgrCd       : \'v1e004003\',\n\t\tselStatCd        : \'103\',\n\t\timgUrl           : \'/t/080/pd/19/2/7/6/4/7/6/YoVtB/2064276476_B.jpg\',\n\t\ttourIframeAcssYn : false,\n\t\tprdDtlTypCd      : "",\n\t\tdlvCstInstB

In [139]:
#course_list = json.loads(json_string)